In [1]:
# Install required libraries
!pip install librosa transformers

In [2]:
# Define paths to data
cha_path = '/kaggle/input/pittcombined/PittCombined/cha'
mp3_path = '/kaggle/input/pittcombined/PittCombined/mp3'

In [3]:
# Import necessary libraries
import os
import librosa
import pandas as pd
from transformers import BertTokenizer, BertModel


In [4]:
# List files in directories
cha_files = [os.path.join(cha_path, file) for file in os.listdir(cha_path) if file.endswith('.cha')]
mp3_files = [os.path.join(mp3_path, file) for file in os.listdir(mp3_path) if file.endswith('.mp3')]

print(f"Found {len(cha_files)} CHA files and {len(mp3_files)} MP3 files.")

Found 1255 CHA files and 1253 MP3 files.


In [5]:
import os
import re
import random
from collections import defaultdict, Counter

def extract_diagnosis(cha_file):
    """Extracts the diagnosis from a CHAT transcript file."""
    with open(cha_file, 'r') as file:
        content = file.read()
    match = re.search(r'@ID:\s*[^|]*\|[^|]*\|[^|]*\|[^|]*\|[^|]*\|([^|]*)\|', content)
    return match.group(1) if match else None

# Paths to your CHA and MP3 files
cha_path = '/kaggle/input/pittcombined/PittCombined/cha'
mp3_path = '/kaggle/input/pittcombined/PittCombined/mp3'

# Load all CHA files and their diagnoses
cha_files = [f for f in os.listdir(cha_path) if f.endswith('.cha')]
diagnoses = [extract_diagnosis(os.path.join(cha_path, f)) for f in cha_files]

# Group files by diagnosis, ensuring corresponding MP3 exists
files_by_diagnosis = defaultdict(list)
for cha_file, diag in zip(cha_files, diagnoses):
    mp3_file = cha_file.replace('.cha', '.mp3')
    if os.path.exists(os.path.join(mp3_path, mp3_file)):
        files_by_diagnosis[diag].append(cha_file)

# Select up to 500 files, trying to balance across diagnoses
selected_files = []
for diag, files in files_by_diagnosis.items():
    select_count = min(len(files), max(500 // len(files_by_diagnosis), 1))
    selected_files.extend(random.sample(files, select_count))

# Ensure the selection does not exceed 10 if categories were unbalanced
selected_files = random.sample(selected_files, min(500, len(selected_files)))

# Count of selected diagnoses
selected_diagnoses = [extract_diagnosis(os.path.join(cha_path, f)) for f in selected_files]
diagnosis_count = Counter(selected_diagnoses)

# Find corresponding MP3 files
selected_mp3_files = [f.replace('.cha', '.mp3') for f in selected_files]

# Collect full paths for the selected files
cha_files = [os.path.join(cha_path, f) for f in selected_files]
mp3_files = [os.path.join(mp3_path, f) for f in selected_mp3_files]

# Output results
print("Selected CHA files:", cha_files)
print("Selected MP3 files:", mp3_files)
print("Diagnosis counts:", dict(diagnosis_count))


Selected CHA files: ['/kaggle/input/pittcombined/PittCombined/cha/024-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/016-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/358-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/689-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/065-2f.cha', '/kaggle/input/pittcombined/PittCombined/cha/471-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/010-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/212-1r.cha', '/kaggle/input/pittcombined/PittCombined/cha/361-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/049-1f.cha', '/kaggle/input/pittcombined/PittCombined/cha/222-1f.cha', '/kaggle/input/pittcombined/PittCombined/cha/051-1r.cha', '/kaggle/input/pittcombined/PittCombined/cha/033-2r.cha', '/kaggle/input/pittcombined/PittCombined/cha/024-2f.cha', '/kaggle/input/pittcombined/PittCombined/cha/145-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/244-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/062-0.cha',

In [6]:
cha_base_names = set([os.path.splitext(os.path.basename(f))[0] for f in cha_files])
mp3_base_names = set([os.path.splitext(os.path.basename(f))[0] for f in mp3_files])

unmatched_cha = cha_base_names - mp3_base_names
unmatched_mp3 = mp3_base_names - cha_base_names

print("Unmatched CHA files:", unmatched_cha)
print("Unmatched MP3 files:", unmatched_mp3)

Unmatched CHA files: set()
Unmatched MP3 files: set()


In [7]:
import librosa
from transformers import BertTokenizer, BertModel
import soundfile as sf

# Initialize BioBERT
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = BertModel.from_pretrained("dmis-lab/biobert-v1.1")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
import os
import librosa
import soundfile as sf

def preprocess_audio(mp3_file_path, output_wav_path):
    # Define output path within the writable directory
    output_wav_path = os.path.join('/kaggle/working', os.path.basename(output_wav_path))
    
    try:
        # Check if the WAV file already exists
        if not os.path.exists(output_wav_path):
            # Convert MP3 to WAV
            y, sr = librosa.load(mp3_file_path, sr=None)
            sf.write(output_wav_path, y, sr)
            print(f"Converted {mp3_file_path} to WAV.")
        else:
            print(f"WAV file already exists: {output_wav_path}")

        # Extract MFCC features
        y, sr = librosa.load(output_wav_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return mfccs.mean(axis=1)
    except Exception as e:
        print(f"An error occurred while processing {mp3_file_path}: {str(e)}")
        return None

# Replace 'mp3_files' with the actual list of mp3 file paths
audio_features = [preprocess_audio(f, f.replace('.mp3', '.wav')) for f in mp3_files if f.endswith('.mp3')]


Converted /kaggle/input/pittcombined/PittCombined/mp3/024-2.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/016-0.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/358-0r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/689-0f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/065-2f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/471-0f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/010-2.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/212-1r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/361-0f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/049-1f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/222-1f.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/051-1r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/033-2r.mp3 to WAV.
Converted /kaggle/input/pittcombined/PittCombined/mp3/

In [9]:
# Define a function to preprocess text data
def preprocess_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    # Insert text cleaning code here as necessary
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    outputs = model(**inputs)
    return outputs.last_hidden_state.squeeze().detach().numpy()

In [10]:
print(cha_files)

['/kaggle/input/pittcombined/PittCombined/cha/024-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/016-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/358-0r.cha', '/kaggle/input/pittcombined/PittCombined/cha/689-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/065-2f.cha', '/kaggle/input/pittcombined/PittCombined/cha/471-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/010-2.cha', '/kaggle/input/pittcombined/PittCombined/cha/212-1r.cha', '/kaggle/input/pittcombined/PittCombined/cha/361-0f.cha', '/kaggle/input/pittcombined/PittCombined/cha/049-1f.cha', '/kaggle/input/pittcombined/PittCombined/cha/222-1f.cha', '/kaggle/input/pittcombined/PittCombined/cha/051-1r.cha', '/kaggle/input/pittcombined/PittCombined/cha/033-2r.cha', '/kaggle/input/pittcombined/PittCombined/cha/024-2f.cha', '/kaggle/input/pittcombined/PittCombined/cha/145-1.cha', '/kaggle/input/pittcombined/PittCombined/cha/244-0.cha', '/kaggle/input/pittcombined/PittCombined/cha/062-0.cha', '/kaggle/input/pitt

In [11]:
# Process each file
text_embeddings = [preprocess_text(f) for f in cha_files]

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming text_embeddings and audio_features are lists of numpy arrays
# Convert lists to numpy arrays
text_features = np.array(text_embeddings)
audio_features = np.stack(audio_features)

In [13]:
print(text_features)

[[[-7.13483542e-02  2.82713324e-01 -6.21027410e-01 ... -9.73568112e-02
    3.11696559e-01  3.42734277e-01]
  [-1.47908609e-02  4.03723300e-01  2.43561793e-04 ...  6.10847950e-01
    8.03448379e-01  2.00707287e-01]
  [ 6.50603846e-02 -5.99107472e-03 -3.39761287e-01 ...  4.41388369e-01
   -1.48363665e-01  2.62811601e-01]
  ...
  [ 7.75481537e-02 -3.70662957e-02  4.15178150e-01 ... -4.20797586e-01
    5.57485782e-02  1.79835677e-01]
  [-6.69489754e-03  1.87827677e-01 -1.65590361e-01 ... -1.90454155e-01
    8.48207846e-02  2.09880993e-01]
  [-2.83187982e-02  5.56173660e-02 -5.59570432e-01 ... -9.30020809e-02
    6.47520423e-01  1.11799456e-01]]

 [[-2.73890961e-02  2.93343477e-02 -5.37693441e-01 ... -2.98129559e-01
    2.37378031e-01  4.05238837e-01]
  [-4.47471552e-02  4.42409217e-01  1.71069264e-01 ...  5.93660414e-01
    6.57433510e-01  2.98134983e-01]
  [ 6.61202222e-02  1.96762569e-03 -3.50829780e-01 ...  3.73287350e-01
   -1.26769379e-01  2.19737381e-01]
  ...
  [ 3.65918665e-03 -3.8

In [14]:
print(audio_features)

[[-4.4555786e+02  1.2426113e+02 -3.8871815e+00 ... -4.6226821e+00
   2.1218130e-01  1.4672548e+00]
 [-5.6964178e+02  1.3796545e+02  6.5924993e+00 ...  4.6396108e+00
  -6.9134507e+00  8.5007010e+00]
 [-5.6910779e+02  8.0828568e+01  2.9009665e+01 ...  5.5039048e-02
   7.1197921e-01 -2.2626984e+00]
 ...
 [-4.2639081e+02  6.5972229e+01  2.3582987e+01 ... -3.4135962e-01
   9.6120149e-02  3.9023079e-02]
 [-5.1181885e+02  1.0809120e+02  3.7009454e+00 ... -7.7031212e+00
  -1.4701397e+00 -8.6570625e+00]
 [-5.3538702e+02  6.2022930e+01  1.6640188e+01 ...  6.2500685e-01
   4.2632055e+00  9.5114791e-01]]


In [15]:
text_features = np.mean(text_features, axis=1)


In [16]:
print(text_features)

[[ 0.14496957 -0.11411361 -0.19778547 ...  0.11795184  0.1790378
   0.2604128 ]
 [ 0.11192731 -0.17277272 -0.06639167 ...  0.07837552  0.2426483
   0.2220686 ]
 [ 0.18740469 -0.12851706 -0.09251877 ...  0.10202659  0.2101165
   0.14520437]
 ...
 [ 0.23411773  0.13274135 -0.08172235 ...  0.16492993  0.33674198
  -0.05822227]
 [ 0.01766926 -0.19782738 -0.21635407 ...  0.07632968  0.11898068
   0.20865297]
 [ 0.09021799 -0.3945113  -0.02100907 ...  0.30115792  0.0469407
   0.05937409]]


In [17]:
# Ensure audio_features is 2D (it should already be if you've extracted features correctly)
if audio_features.ndim > 2:
    audio_features = np.mean(audio_features, axis=1)

In [18]:
# Combine text and audio features
combined_features = np.concatenate([text_features, audio_features], axis=1)

In [19]:
print(combined_features)

[[ 0.14496957 -0.11411361 -0.19778547 ... -4.622682    0.2121813
   1.4672548 ]
 [ 0.11192731 -0.17277272 -0.06639167 ...  4.639611   -6.9134507
   8.500701  ]
 [ 0.18740469 -0.12851706 -0.09251877 ...  0.05503905  0.7119792
  -2.2626984 ]
 ...
 [ 0.23411773  0.13274135 -0.08172235 ... -0.34135962  0.09612015
   0.03902308]
 [ 0.01766926 -0.19782738 -0.21635407 ... -7.703121   -1.4701397
  -8.657063  ]
 [ 0.09021799 -0.3945113  -0.02100907 ...  0.62500685  4.2632055
   0.9511479 ]]


In [20]:
# Normalize the combined features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(combined_features)

In [21]:
import re

def extract_diagnostic_code(cha_file):
    # Read the content of the .cha file
    with open(cha_file, 'r') as file:
        content = file.read()

    # Regex to find the diagnosis in the file content based on the updated structure
    match = re.search(r'@ID:\s*[^|]*\|[^|]*\|[^|]*\|[^|]*\|[^|]*\|([^|]*)\|\|', content)
    if match:
        return match.group(1).strip()
    return None

labels = [extract_diagnostic_code(f) for f in cha_files]

for label in labels[:10]:  # Print first 10 labels to verify
    print(label)


Memory
MCI
Probable
Vascular
MCI
PossibleAD
ProbableAD
MCI
PossibleAD
ProbableAD


In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [23]:
print(encoded_labels)

[3 2 6 8 2 5 7 2 5 7 7 7 2 3 0 7 2 3 5 2 2 2 5 5 7 3 0 7 0 2 2 7 5 5 0 5 5
 7 7 5 0 0 8 0 8 0 7 0 5 7 5 2 0 7 8 0 5 8 5 5 7 5 5 2 7 0 7 7 8 0 4 5 5 0
 7 8 5 7 0 0 5 5 0 5 5 7 2 8 2 0 2 3 2 2 0 0 7 2 2 7 0 7 7 5 2 0 5 7 2 2 2
 7 6 2 3 5 8 7 7 5 1 5 2 7 5 5 8 5 0 6 5 2 7 7 8 0 7 2 7 7 5 3 0 7 2 5 5 2
 8 0 7 0 2 3 0 7 0 5 0 0 2 2 2 7 5 7 0 7 0 4 0 8 3 6 7 7 7 0 7 3 5 0 7 5 2
 2 0 7 2 0 8 2 0 2 2 2 5 5 5 2 5 8 5 7 0 2 2 2 0 0 0 2 7 5 2 5 5 2 2 5 0 7
 2 0 7 5 2 2 5 8 0 7 8 3 5 8 2 8 0 7 4 7 0 0 2 5 2 3 2 0 0 0 0 5 4 7 0 5 0
 7]


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Conv1D, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam

2024-08-14 21:49:56.123534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 21:49:56.123741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 21:49:56.327241: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [26]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(normalized_features, encoded_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Finding the maximum label value across all datasets to ensure consistency
max_label = max(np.max(y_train), np.max(y_val), np.max(y_test)) + 1  # Plus one because classes are zero-indexed

# One-hot encode the labels with a consistent number of classes across datasets
y_train = to_categorical(y_train, num_classes=max_label)
y_val = to_categorical(y_val, num_classes=max_label)
y_test = to_categorical(y_test, num_classes=max_label)




In [27]:
print("y_test",y_test)

y_test [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0

In [28]:
from tensorflow.keras.regularizers import l2

# Model architecture
input_text = Input(shape=(768,))  # Text features from Clinical BERT
input_audio = Input(shape=(13,))  # Audio features, assuming MFCCs with 13 coefficients

# Text pathway
text_dense = Dense(128, activation='relu', kernel_regularizer=l2(0.1))(input_text)
text_bn = BatchNormalization()(text_dense)  # Add batch normalization
text_out = Dropout(0.5)(text_bn)  # Increase dropout rate


In [29]:
from tensorflow.keras.layers import Reshape

# Audio pathway
audio_reshape = Reshape((13, 1))(input_audio)
conv1 = Conv1D(64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.1))(audio_reshape)
conv1_bn = BatchNormalization()(conv1)
conv1_pool = GlobalAveragePooling1D()(conv1_bn)
audio_out = Dropout(0.5)(conv1_pool)  # Increase dropout rate

In [30]:
# Fusion and output
concatenated = Concatenate()([text_out, audio_out])
dense_layer = Dense(64, activation='relu', kernel_regularizer=l2(0.1))(concatenated)  # Regularize dense layer

# Assuming y_train has already been one-hot encoded correctly
num_classes = y_train.shape[1]

# Adjust your output layer
output_layer = Dense(num_classes, activation='softmax')(dense_layer)


In [31]:
print(concatenated)

<KerasTensor shape=(None, 192), dtype=float32, sparse=False, name=keras_tensor_10>


In [32]:
from tensorflow.keras.metrics import Precision, Recall

# Compile model
model = Model(inputs=[input_text, input_audio], outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])


In [33]:
# Fit model on training data
model.fit([X_train[:, :768], X_train[:, 768:]], y_train, validation_data=([X_val[:, :768], X_val[:, 768:]], y_val), epochs=100, batch_size=32)


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - accuracy: 0.1631 - loss: 33.7319 - precision: 0.1545 - recall: 0.0271 - val_accuracy: 0.3269 - val_loss: 31.5204 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3830 - loss: 30.6741 - precision: 0.6585 - recall: 0.1309 - val_accuracy: 0.4423 - val_loss: 28.7667 - val_precision: 0.5000 - val_recall: 0.0385
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3269 - loss: 28.0415 - precision: 0.4801 - recall: 0.1215 - val_accuracy: 0.4423 - val_loss: 26.2155 - val_precision: 0.8333 - val_recall: 0.0962
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4580 - loss: 25.3731 - precision: 0.6579 - recall: 0.2050 - val_accuracy: 0.5000 - val_loss: 23.8287 - val_precision: 0.5556 - val_recall: 0.0962
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5878 - loss: 22.8710 - precision: 0.7458 - recall: 0.2670 - val_accuracy: 0.5385 - val_

In [34]:
# Evaluate the model on the test set
evaluation = model.evaluate([X_test[:, :768], X_test[:, 768:]], y_test)
print(f'Accuracy: {evaluation[1]*100:.2f}%, Precision: {evaluation[2]*100:.2f}%, Recall: {evaluation[3]*100:.2f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5280 - loss: 1.7503 - precision: 0.5813 - recall: 0.4071
Accuracy: 55.77%, Precision: 61.11%, Recall: 42.31%
